In [20]:
import pandas as pd
import numpy as n

In [21]:
# Cargar los CSVs procesados
df_customer = pd.read_csv('../data/customer_clean.csv')
df_store = pd.read_csv('../data/store_clean.csv')
df_rental = pd.read_csv('../data/rental_clean.csv')
df_inventory = pd.read_csv('../data/inventory_clean.csv')
df_film = pd.read_csv('../data/film_clean.csv')

In [22]:
# ---- Análisis ----
# Total de alquileres por tienda
rental_inventory = df_rental.merge(df_inventory, on="inventory_id", how="inner")
rental_store = rental_inventory.merge(df_store, on="store_id", how="inner")
rentals_by_store = rental_store.groupby("store_id").size().reset_index(name="total_rentals")
rental_film = rental_inventory.merge(df_film, on="film_id", how="inner")

In [25]:

# ---- Total de alquileres por género ----
# Usar 'rating' como una aproximación de género
if 'rating' in df_film.columns:
    rental_film = rental_inventory.merge(df_film, on="film_id", how="inner")
    rentals_by_genre = rental_film.groupby("rating").size().reset_index(name="total_rentals_by_genre")
    print("\nTotal de alquileres por género:")
    print(rentals_by_genre)
else:
    print("\nLa columna 'rating' no está disponible en df_film. No se puede calcular el total de alquileres por género.")



Total de alquileres por género:
   rating  total_rentals_by_genre
0       G                    2740
1   NC-17                    3249
2      PG                    3177
3   PG-13                    3612
4       R                    3152


In [26]:
# ---- Total de alquileres por año de lanzamiento ----
if 'release_year' in df_film.columns:
    rentals_by_year = rental_film.groupby("release_year").size().reset_index(name="total_rentals_by_year")
    print("\nTotal de alquileres por año de lanzamiento:")
    print(rentals_by_year)
else:
    print("\nLa columna 'release_year' no está disponible en df_film. No se puede calcular el total de alquileres por año de lanzamiento.") 



Total de alquileres por año de lanzamiento:
   release_year  total_rentals_by_year
0          2006                  15930


In [30]:
# ---- Duración promedio de los alquileres ----
df_rental['rental_date'] = pd.to_datetime(df_rental['rental_date'])
df_rental['return_date'] = pd.to_datetime(df_rental['return_date'])
df_rental['rental_duration_days'] = (df_rental['return_date'] - df_rental['rental_date']).dt.days
avg_duration = df_rental['rental_duration_days'].mean()
print(f"\nDuración promedio de los alquileres: {avg_duration:.2f} días")

# ---- Películas más alquiladas ----
if 'title' in df_film.columns:
    most_rented_films = rental_film.groupby(["film_id", "title"]).size().reset_index(name="total_rentals_by_film").sort_values("total_rentals_by_film", ascending=False).head(10)
    print("\nPelículas más alquiladas:")
    print(most_rented_films)
else:
    print("\nLa columna 'title' no está disponible en df_film. No se puede calcular las películas más alquiladas.")


Duración promedio de los alquileres: 4.53 días

Películas más alquiladas:
     film_id                   title  total_rentals_by_film
302      320   FLAMINGOS CONNECTICUT                     92
96       103      BUCKET BROTHERHOOD                     34
705      738        ROCKETEER MOTHER                     33
361      382          GRIT CLOCKWORK                     32
312      331          FORWARD TEMPLE                     32
733      767           SCALAWAG DUCK                     32
29        31           APACHE DIVINE                     31
719      753         RUSH GOODFELLAS                     31
702      735            ROBBERS JOON                     31
930      973               WIFE TURN                     31


In [31]:
# Relación entre costo de reemplazo y popularidad
if "replacement_cost" in df_film.columns:
    # Join entre rental y film para obtener popularidad
    rental_film = df_rental.merge(df_inventory, on="inventory_id", how="inner") \
                           .merge(df_film, on="film_id", how="inner")
    
    # Calcular total de alquileres por película y reemplazo
    film_popularity = rental_film.groupby(["film_id", "replacement_cost"]).size().reset_index(name="total_rentals_by_film")
    
    # Calcular correlación entre 'replacement_cost' y 'total_rentals_by_film'
    correlation = film_popularity["replacement_cost"].corr(film_popularity["total_rentals_by_film"])
    
    print(f"Correlación entre 'replacement_cost' y 'total_rentals_by_film': {correlation}")
else:
    print("La columna 'replacement_cost' no está disponible en df_film.")

Correlación entre 'replacement_cost' y 'total_rentals_by_film': 0.10213236883598363
